In [22]:
!pip3 install mysql-connector-python python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [23]:
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv
import os

In [24]:
# Load environment variables from .env file
load_dotenv()

# Database connection details
db_host = 'localhost'
db_user = os.getenv('MYSQL_USER')
db_password = os.getenv('MYSQL_PASSWORD')
db_database = os.getenv('MYSQL_DATABASE')

In [25]:
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

In [26]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Query executed successfully: {query}")
    except Error as e:
        print(f"The error '{e}' occurred when executing: {query}")

In [27]:
def execute_script_from_file(connection, file_path):
    cursor = connection.cursor()
    with open(file_path, 'r') as file:
        script = file.read()

    statements = script.split(';')

    for statement in statements:
        if statement.strip():
            try:
                cursor.execute(statement)
                connection.commit()
                print(f"Statement executed successfully: {statement}")
            except Error as e:
                if e.errno == 1050:  # Error code for "Table already exists"
                    print(f"The error '{e}' occurred when executing statement: {statement} - Table already exists, continuing...")
                elif e.errno == 1061:  # Error code for "Duplicate key name"
                    print(f"The error '{e}' occurred when executing statement: {statement} - Duplicate key name, continuing...")
                elif e.errno == 1062:  # Error code for "Duplicate entry"
                    print(f"The error '{e}' occurred when executing statement: {statement} - Duplicate entry, continuing...")
                elif e.errno == 1146:  # Error code for "Table doesn't exist"
                    print(f"The error '{e}' occurred when executing statement: {statement} - Table doesn't exist, continuing...")
                else:
                    print(f"The error '{e}' occurred when executing statement: {statement}")

In [28]:
def main():
    connection = create_connection(db_host, db_user, db_password, db_database)

    if connection:
        execute_script_from_file(connection, "1create.sql")
        execute_script_from_file(connection, "2contraines.sql")
        execute_script_from_file(connection, "3insert.sql")

        connection.close()

if __name__ == "__main__":
    main()

Connection to MySQL DB successful
The error '1050 (42S01): Table 'Clients' already exists' occurred when executing statement: CREATE TABLE Clients (
    id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(50) NOT NULL UNIQUE,
    email VARCHAR(100) NOT NULL UNIQUE,
    password VARCHAR(255) NOT NULL,
    role ENUM('admin', 'client') NOT NULL DEFAULT 'client',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
) - Table already exists, continuing...
The error '1061 (42000): Duplicate key name 'unique_username'' occurred when executing statement: ALTER TABLE Clients
ADD CONSTRAINT unique_username UNIQUE (username),
ADD CONSTRAINT unique_email UNIQUE (email) - Duplicate key name, continuing...
The error '1062 (23000): Duplicate entry 'user' for key 'username'' occurred when executing statement: INSERT INTO Clients (username, email, password, role)
VALUES ('user', 'u@u.com', PASSWORD('1234'), 'client') - Du